In [ ]:
Title: My Title
Authors : 
  "Author1": 
  "Author2":

In [ ]:
import numpy as np
import pandas as pd


import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt


import random 
from tqdm.auto import tqdm

import warnings
%matplotlib inline
warnings.filterwarnings('ignore')

<hr>
<h2>Description</h2>
<p>Le jeu de données contenu dans train.csv décrit des candidatures au poste de
chercheur d’or chez OrFée.<br>

<p>Le jeu de données comporte 12 colonnes :<br>
• <b>id</b>  : identifiant unique de chaque ligne<br>
• <b>date</b>  : date de la candidature<br>
•  <b>age</b>: âge du candidat<br>
•  <b>diplome</b>: plus haut diplôme obtenu (bac, licence, master, doctorat)<br>
• <b>specialite</b> : spécialité du diplôme (géologie, forage, détective, archéologie, . . . )<br>
• <b>salaire</b>: salaire demandé<br>
• <b>dispo</b>: oui : disponibilité immédiate, non : pas disponible immédiatement<br>
• <b>sexe</b>: féminin (F) ou masculin (M)<br>
• <b>exp</b> : nombre d’années d’expérience<br>
• <b>cheveux</b> : couleur des cheveux (châtain, brun, blond, roux)<br>
• <b>note</b> : note (sur 100) obtenue à l’exercice de recherche d’or<br>
• <b>embauche</b> : le candidat a-t-il été embauché ? (0 : non, 1 : oui) <br>

In [ ]:
df = pd.read_csv(r'../data/data.csv',sep=';')
df.shape

In [ ]:
df.sample(5)

In [ ]:
df.columns

# <span style="color:#011C5D">Analyse descriptive </span>

In [ ]:
df.info()

### <span style="color:#011C5D">1. Identifier les variables numeriques et categorielles </span>

In [ ]:
# EXERCICE : 

# Identifions tout d'abord les colonnes catégorielles
col_categoricals = None

# Puis les numériques
col_numericals = None


print(f"Liste des colonnes catégorielles :\n{col_categoricals}")
print(f"\nListe des colonnes numériques :\n{col_numericals}")

In [ ]:
# Convertir les données numerique au bon format "SI BESOIN"
#### CODEZ-ICI ####

### <span style="color:#011C5D">2. Analyse de la distribution des variables numeriques </span>

In [ ]:
ncols = 2
nrows = len(col_numericals) // ncols + (len(col_numericals) % ncols > 0)
 
plt.figure(figsize=(17, 14))
plt.subplots_adjust(hspace=1)
plt.suptitle("Distribution des \nvariables continues", fontsize=18, y=0.95)
 
for n, variable in enumerate(col_numericals):
    # add a new subplot iteratively
    ax = plt.subplot(nrows, ncols, n + 1)
 
    sns.kdeplot(ax=ax, data=df[variable], legend=None, shade=True)
    ax.grid(b=True, which='major', axis='y')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    
    ax.set_title(variable, loc= 'center', fontsize=12)

### <span style="color:#011C5D">3. Matrice de correlation </span>

In [ ]:
# EXERCICE : 

# Calculer la correlation entre les variables numeriques

matrice_corr = None

# Afficher un heatmap de la matrice de correlation 



# Que pouvez-vous conclure ?

### <span style="color:#011C5D">4. Identification des valeurs manquantes </span>

In [ ]:
# EXERCICE : Laissez libre court à votre créativité et créez une fonction qui affiche le taux
#            de valeurs manquantes de chaque variable pour chaque type (catégorielle et numérique)
#            par ordre décroissant


def display_missing_values(df):
    
#### CODEZ-ICI ####



# Que pouvez-vous conclure ?

### <span style="color:#011C5D">5. Imputation des valeurs manquantes </span>

#### <span style="color:#011C5D">5.1 Variables numeriques </span>

##### <span style="color:#011C5D">5.1.1 Imputation par la moyenne </span>

In [ ]:
# EXERCICE : Imputation des variables numériques avec la moyenne
# Les variables imputées sont enregistrées dans de nouvelles colonnes.



##### <span style="color:#011C5D">5.1.2 Imputation par la regression </span>

In [ ]:
#EXERCICE : Imputation par régression
# Laissez libre cours à votre créativité !
# Pour chaque variable numérique contenant des valeurs manquantes, utilisez les variables explicatives appropriées afin d’imputer les valeurs manquantes à l’aide d’une méthode de régression.

##### <span style="color:#011C5D">5.1.3 Comparer la distribution des variables imputer </span>

In [ ]:
# EXERCICE : Laissez libre court à votre créativité 
# Pour chaque variable numerique
# Comparer la distribution entre l'imputation par la moyenne et l'imputation par Regression



In [ ]:
# Que pouvez-vous conclure ?

#### <span style="color:#011C5D">5.2 Variables categorielles </span>

In [ ]:
# EXERCICE : Utiliser une methode adequate pour imputer les variables categorielles
# Les variables imputées sont enregistrées dans de nouvelles colonnes.



### <span style="color:#011C5D">6. Detection des outliers </span>

In [ ]:
# On affiche les boxplots
temp=pd.melt(df[col_numericals].reset_index(), id_vars='index', value_vars=col_numericals).iloc[:,-2:]
 
ncols = 2
nrows = len(col_numericals) // ncols + (len(col_numericals) % ncols > 0)
 
plt.figure(figsize=(17, 14))
plt.subplots_adjust(hspace=1)
plt.suptitle("Boxplot des \nvariables continues", fontsize=18, y=0.95)
 
for n, variable in enumerate(col_numericals):
    # add a new subplot iteratively
    ax = plt.subplot(nrows, ncols, n + 1)
 
    sns.boxplot(ax=ax, data=temp[temp['variable']==variable], orient='h')
    #sns.kdeplot(ax=ax, data=temp[temp['variable']==variable], legend=None, palette=customPalette, shade=True)
    ax.grid(b=True, which='major', axis='y')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    
    ax.set_title(variable, loc= 'center', fontsize=12)

In [ ]:
# EXERCICE : Completer cette fonction qui retourne 0 ou 1 si la valeur observée est outlier ou pas 

def is_outlier(df,column) :
    
    # 1er Quartile 
    Q1 = 
    
    # 3ème Quartile 
    Q3 = 
    
    # Inter-Quartile Range (IQR)
    IQR = Q3 - Q1
    
    # limites, basse & haute
    limite_inf = 
    limte_sup = 
    
    # Remplace les données inférieur et supérieur à la limite par 1 et les autres par 0
    series = 
    
    return series

In [ ]:
# EXERCICE : Appliquer votre fonction aux variables numeriques




In [ ]:
# Que pouvez-vous conclure ?

In [ ]:
# EXERCICE : Laissez libre court à votre créativité et créez une fonction qui affiche le taux et nombre
#            de valeurs manquantes de chaque variable
#            par ordre décroissant



### <span style="color:#011C5D">7. Encodage des variables categorielles </span>

In [ ]:
# EXERCICE : Utiliser une methode adequate pour encoder les variables categorielles



### <span style="color:#011C5D">8. Encodage des variables categorielles Embedding </span>

In [ ]:
import torch
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Exemple variable
VARIABLE = "cheveux"
df[VARIABLE] = df[VARIABLE].fillna("Missing")
MODALITE = df[VARIABLE].unique()
NB_MODALITE = len(MODALITE)
NB_MODALITE,MODALITE

In [ ]:
DIM_EMBEDDING = 10

In [ ]:
# Exemple d'embedding avec des weights aleatoire non entraine sur une dataset
embedding = nn.Embedding(NB_MODALITE, DIM_EMBEDDING)

In [ ]:
df[VARIABLE+"_label_encoded"] = LabelEncoder().fit_transform(df[VARIABLE])

In [ ]:
df[[VARIABLE,VARIABLE+"_label_encoded"]].head()

In [ ]:
variable_to_torch = torch.tensor(df[VARIABLE+"_label_encoded"].values.tolist())  

In [ ]:
embeddings = embedding(variable_to_torch)

In [ ]:
embeddings

In [ ]:
embeddings.shape

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [ ]:
### Visualisation des embeddings
embeddings_weight = embedding.weight.data.numpy()
pca = PCA(n_components=2)
embeddings_pca = pca.fit_transform(embeddings_weight)


plt.scatter(embeddings_pca[:, 0], embeddings_pca[:, 1])
plt.show()

### <span style="color:#011C5D">9. Créer votre package python </span>

    - Step 1 : Créer un fichier analyse_exploratoire.py
    - Step 2 : Copier toutes les fonctions de ce notebook dans votre fichier fichier analyse_exploratoire.py
    - Step 3 : Créer un dossier src
    - Step 4 : Deplacer votre fichier analyse_exploratoire.py dans le dossier src
    - Step 5 : Créer un nouveau notebook
    - Step 6 : Dans ce notebook, specifier le chemin vers votre dossier src
    - Step 7 : Executer 'from  analyse_exploratoire import is_outlier, display_missing_values etc...
    - Step 8 : Analyser vos données avec ces nouvelles fonction

## Ressources

Imputation:

- [Imputation Iterrative](https://scikit-learn.org/stable/auto_examples/impute/plot_iterative_imputer_variants_comparison.html) 
- [Imputation](https://scikit-learn.org/stable/modules/impute.html) 

Outliers:

- [Doc sklearn](https://scikit-learn.org/stable/modules/outlier_detection.html)
- [Doc Medium](https://towardsdatascience.com/5-outlier-detection-methods-that-every-data-enthusiast-must-know-f917bf439210)